<a href="https://colab.research.google.com/github/soadD/netflix_data/blob/main/tabela_global_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Extração**

In [ ]:
#Abertura das bibliotecas
import pandas as pd
#nova modificacao ewerton after gitignore

In [ ]:
# Importar a biblioteca para conectar o Google Drive ao ambiente Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Passo 4: Extração da informação via pandas
df = pd.read_csv('/content/drive/MyDrive/Soul Code/netflix/netflixtop10.csv')

In [ ]:
display(df)

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,runtime,weekly_views,cumulative_weeks_in_top_10,is_staggered_launch,episode_launch_details
0,2024-05-26,Films (English),1,Atlas,NaN,56300000,2,28200000.0,1,False,NaN
1,2024-05-26,Films (English),2,Thelma the Unicorn,NaN,17400000,"1,6333",10700000.0,2,False,NaN
2,2024-05-26,Films (English),3,Ice Age: Dawn of the Dinosaurs,NaN,15100000,"1,5833",9500000.0,1,False,NaN
3,2024-05-26,Films (English),4,Mother of the Bride,NaN,14000000,"1,5",9300000.0,3,False,NaN
4,2024-05-26,Films (English),5,Disturbia,NaN,14100000,"1,75",8100000.0,2,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6075,2021-07-04,TV (Non-English),6,Elite,Elite: Season 1,10530000,NaN,NaN,1,False,NaN
6076,2021-07-04,TV (Non-English),7,Elite,Elite: Season 3,10200000,NaN,NaN,1,False,NaN
6077,2021-07-04,TV (Non-English),8,Elite,Elite: Season 2,10140000,NaN,NaN,1,False,NaN
6078,2021-07-04,TV (Non-English),9,Katla,Katla: Season 1,9190000,NaN,NaN,1,False,NaN


## **Pré-Análise**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        6080 non-null   object 
 1   category                    6080 non-null   object 
 2   weekly_rank                 6080 non-null   int64  
 3   show_title                  6080 non-null   object 
 4   season_title                2959 non-null   object 
 5   weekly_hours_viewed         6080 non-null   int64  
 6   runtime                     2000 non-null   object 
 7   weekly_views                2000 non-null   float64
 8   cumulative_weeks_in_top_10  6080 non-null   int64  
 9   is_staggered_launch         6080 non-null   bool   
 10  episode_launch_details      78 non-null     object 
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 481.1+ KB


Colunas 'episode_launch_details','season_title','weekly_views' e 'runtime' não tem dados suficientes para fazer análises.

Coluna week é uma série temporal, porém está em object (string) ao invés de date time.


## **Transformação**

### **Exclusão de Colunas**

In [ ]:
#Exclusão de colunas (números elevados de valores nulos)
lista = ['episode_launch_details','season_title','weekly_views','runtime','is_staggered_launch']

for i in lista:
  df.drop(i, axis=1, inplace=True)

In [ ]:
# Renomeando colunas para tradução
df.rename(columns={'week' : 'semana',
                   'category' : 'categoria',
                   'weekly_rank' : 'classificaçao_semanal',
                   'show_title' : 'titulo_do_programa/show',
                   'season_title' : 'titulo da temporada',
                   'weekly_hours_viewed' : 'horas_visualizadas',
                   'cumulative_weeks_in_top_10' : 'semanas_acumuladas_top10'}, inplace=True)

### **Verificação de valores nulos**

In [ ]:
# Verificando valores null
df.isnull().sum()

semana                      0
categoria                   0
classificaçao_semanal       0
titulo_do_programa/show     0
horas_visualizadas          0
semanas_acumuladas_top10    0
dtype: int64

In [ ]:
# Verificando valores nan
df.isna().sum()

semana                      0
categoria                   0
classificaçao_semanal       0
titulo_do_programa/show     0
horas_visualizadas          0
semanas_acumuladas_top10    0
dtype: int64

### **Valores unicos e dados duplicados (REVISAR)**

Como em nossa tabela todos os valores de todas as colunas se repetem eventualmente, é necessário fazer uma verificação de valores duplicados de um mesmo título na mesma semana.

## **Conversão de String para Date Time**

In [ ]:
df['semana'] = pd.to_datetime(df['semana'], infer_datetime_format=True)

<ipython-input-31-7c84dc2c8b97>:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['semana'] = pd.to_datetime(df['semana'], infer_datetime_format=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   semana                    6080 non-null   datetime64[ns]
 1   categoria                 6080 non-null   object        
 2   classificaçao_semanal     6080 non-null   int64         
 3   titulo_do_programa/show   6080 non-null   object        
 4   horas_visualizadas        6080 non-null   int64         
 5   semanas_acumuladas_top10  6080 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 285.1+ KB


In [ ]:
df['mes'] = df['semana'].dt.month
df['ano'] = df['semana'].dt.year
df['dia'] = df['semana'].dt.day

### **Traduzir valores das colunas ???????**